# Демонстратор ИИ-агента для оценки ответов студентов по заданной теме

Создаем ИИ-агент для ПК ИТМО 2025. Агент должен задавать студенту
открытые вопросы по теме (SRE), затем оценивать ответы.

## Установка пакетов и окружения

In [ ]:
# === Установка пакетов (если не установлены) ===
!pip install langchain-gigachat faiss-cpu fastapi uvicorn nest_asyncio google-colab sentence-transformers langchain-huggingface langchain-community

In [ ]:
import os
import json
import nest_asyncio
from typing import List
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
import re
import random # Импортируем random

# --- Импорты Langchain и GigaChat ---
from langchain_gigachat import GigaChat
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# --- Импорт эмбеддингов ---
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
# Вписать путь к директории, где лежит проект
# ИЗМЕНИТЬ ЗДЕСЬ НА СВОЮ ДИРЕКТОРИЮ

ROOT = "/content/drive/MyDrive/AI_agent_ITMO_2025"

In [ ]:
# Подключение к гугл-диску

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Настройки и ключи

In [ ]:
# --- 1. Настройки и ключи ---
# Достаем ключи из "Секреты" Google Colab
# В "Секретах" (ключик слева) Google Colab дожны быть прописаны ключи авторизации
# Ключ Gigachat должен назваться 'SBER_AUTH'
# Ключ Гигачата получить здесь: https://developers.sber.ru/studio/login

from google.colab import userdata

# Попробуем получить токен GigaChat
try:
    GIGA_AUTH = userdata.get('SBER_AUTH')
    if not GIGA_AUTH:
        raise ValueError("❌ Не найден ключ SBER_AUTH в userdata.")
    print("✅ Ключ SBER_AUTH найден.")
except Exception as e:
    print(f"❌ Ошибка при получении SBER_AUTH: {e}")
    # Если ключ не найден, раскомментируйте следующую строку и вставьте ваш ключ
    # GIGA_AUTH = "your_gigachat_auth_key_here"

✅ Ключ SBER_AUTH найден.


## FastAPI

In [ ]:
# --- 2. Инициализация FastAPI ---
app = FastAPI(title="RAG-оценщик с GigaChat (langchain)", version="1.0")

## Создание базы знаний RAG

In [ ]:
# --- 3. Загрузка и индексация документов (RAG) ---

# В папке "AI_agent_ITMO_2025" на гугле диске лежат файлы с теорией и вопросами
# Файл теории - "The_SRE_Lecture.txt",
# файл с вопросами - "The_SRE_questions.txt".
def load_docs(path=ROOT) -> List[LangchainDocument]:
    """Загружает и разбивает документы из указанной директории."""
    # Убедимся, что директория существует
    os.makedirs(path, exist_ok=True)

    # Если директория пуста, создадим тестовые файлы
    if not os.listdir(path):
        print(f"⚠️  Директория {path} пуста. Создаю тестовые файлы.")
        with open(os.path.join(path, "ai_intro.txt"), "w", encoding="utf-8") as f:
            f.write("ИИ — это область информатики, изучающая системы, которые могут учиться, рассуждать и выполнять задачи, требующие интеллекта.")
        with open(os.path.join(path, "oop.txt"), "w", encoding="utf-8") as f:
            f.write("ООП основывается на инкапсуляции, наследовании и полиморфизме. Эти принципы обеспечивают модульность и переиспользование кода.")

    texts = []
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isfile(file_path):
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                    texts.append(LangchainDocument(page_content=content, metadata={"source": file}))
                    print(f"✅ Загружен документ: {file}")
            except UnicodeDecodeError:
                # Попробуем открыть с другой кодировкой, если UTF-8 не сработал
                try:
                    with open(file_path, "r", encoding="cp1251") as f:
                        content = f.read()
                        texts.append(LangchainDocument(page_content=content, metadata={"source": file}))
                        print(f"✅ Загружен документ (cp1251): {file}")
                except UnicodeDecodeError:
                    print(f"❌ Не удалось декодировать файл: {file_path}")
            except Exception as e:
                print(f"❌ Ошибка при чтении файла {file_path}: {e}")

    splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
    split_docs = []
    for doc in texts:
        chunks = splitter.split_documents([doc])
        split_docs.extend(chunks)
    print(f"✅ Разбито на {len(split_docs)} фрагментов.")
    return split_docs

## Парсинг вопросов

In [ ]:
# --- 3.1 Загрузка вопросов из файла на Google Диске ---
def load_questions(file_path) -> List[str]:
    """Загружает список вопросов из указанного файла."""
    questions = []
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            # Простой способ разделить вопросы - по двум и более переносам строк или по нумерации.
            # Этот пример ищет строки, начинающиеся с цифры и точки (например, "1. ", "2. ").
            # Вы можете адаптировать шаблон под формат вашего файла.
            potential_questions = re.split(r'\n\s*\n|\d+\.\s*', content)
            # Отфильтруем пустые строки и строки, которые были просто частью разделителя
            questions = [q.strip() for q in potential_questions if q.strip() and not q.strip().isdigit()]
            # Альтернатива: если вопросы просто разделены переносами строк, используйте:
            # questions = [line.strip() for line in content.splitlines() if line.strip()]
            print(f"✅ Загружено {len(questions)} вопросов из {file_path}")
    except FileNotFoundError:
        print(f"❌ Файл вопросов не найден: {file_path}. Создаю тестовый файл.")
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, "w", encoding="utf-8") as f:
            f.write("1. Что такое SRE?\n\n2. Какие основные принципы SRE?\n\n3. Что такое SLI, SLO и SLA?")
        print("✅ Тестовый файл 'The_SRE_questions.txt' создан.")
        # Рекурсивный вызов для загрузки созданных вопросов
        return load_questions(file_path)
    except Exception as e:
        print(f"❌ Ошибка при чтении файла вопросов {file_path}: {e}")
    return questions



## Загружаем документы и вопросы

In [ ]:
# --- 3.2 Инициализация данных ---
docs = load_docs()
questions_list = load_questions(ROOT + "/The_SRE_questions.txt")

✅ Загружен документ: The_SRE_questions.txt
✅ Загружен документ: The_SRE_Lecture.txt
✅ Разбито на 340 фрагментов.
✅ Загружено 114 вопросов из /content/drive/MyDrive/AI_agent_ITMO_2025/The_SRE_questions.txt


## Gigachat и эмбеддинги

In [ ]:
# --- 4. Инициализация GigaChat и Embeddings ---

# --- 4.1 Инициализация клиента GigaChat для генерации ответов и оценки ---
llm = GigaChat(
    credentials=GIGA_AUTH,
    scope="GIGACHAT_API_PERS",
    model="GigaChat-2-Max",
    temperature=0.2,
    verify_ssl_certs=False
)
print("✅ Инициализирована модель GigaChat")

✅ Инициализирована модель GigaChat


In [ ]:
# --- 4.2 Используем HuggingFaceEmbeddings ---
print("🔄 Загружаю модель эмбеддингов (langchain-huggingface)...")
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
print("✅ Используем HuggingFaceEmbeddings (paraphrase-multilingual-MiniLM-L12-v2).")

🔄 Загружаю модель эмбеддингов (langchain-huggingface)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Используем HuggingFaceEmbeddings (paraphrase-multilingual-MiniLM-L12-v2).


In [ ]:
 #--- 5. Создание векторного хранилища и RAG цепочки (для поиска контекста при оценке) ---
print("🔄 Создаю векторное хранилище FAISS...")
vectorstore = FAISS.from_documents(documents=docs, embedding=embedder)
retriever = vectorstore.as_retriever()
print("✅ Векторное хранилище создано.")

🔄 Создаю векторное хранилище FAISS...
✅ Векторное хранилище создано.


## Промтинг и оценка

In [ ]:
# --- 5. Промпт и функция оценки ---
EVAL_PROMPT_TMPL = """
Ты — преподаватель. Контекст ниже содержит учебные материалы.
Используя этот контекст, оцени ответ студента на вопрос.
Контекст:
{context}

Вопрос: {question}
Ответ студента: {student_answer}

Задачи:
1) Сравни ответ студента с контекстом.
2) Верни только JSON вида:
{{
  "verdict": "верно/частично верно/неверно",
  "feedback": "текст объяснения",
  "relevance": число от 0 до 1
}}
Определения:
- verdict:
  - "верно": Ответ полностью совпадает с контекстом или его логическим следствием.
  - "частично верно": Ответ содержит элементы из контекста, но также включает неточности или неполные объяснения.
  - "неверно": Ответ противоречит контексту, искажает его или не отвечает на суть вопроса. Либо ответ пустой.
- relevance: Число от 0 до 1, отражающее *точность* и *полноту* понимания темы, заданной в Вопросе, на основе Ответа студента. 0.0 - полное непонимание/противоречие, фактическое отсутствие ответа или повтор вопроса, 1.0 - исчерпывающее и точное понимание. Для "неверно" relevance обычно должна быть низкой (например, 0.0 - 0.3), для "частично верно" - средней (0.4 - 0.6), для "верно" - высокой (0.7 - 1.0).

ВАЖНО: Не добавляй комментарии внутри JSON. Комментарии разрешены только до или после JSON-объекта.
"""

def evaluate_answer(query: str, student_answer: str, retriever_instance):
    """Оценивает ответ студента на заданный вопрос, используя контекст из векторного хранилища."""
    import re # Импортируем re в начале функции # <-- ИСПРАВЛЕНО
    # Получаем релевантные документы
    retrieved_docs = retriever_instance.get_relevant_documents(query)
    context = "\n\n".join([d.page_content for d in retrieved_docs])

    # Формируем промпт
    prompt_text = EVAL_PROMPT_TMPL.format(context=context, question=query, student_answer=student_answer)

    # Вызываем LLM
    messages = [{"role": "user", "content": prompt_text}]
    try:
        response = llm.invoke(messages)
        # llm.invoke возвращает AIMessage, получаем content
        raw_response_text = response.content
        print(f"DEBUG: Raw LLM response:\n{raw_response_text}\n---")

        # Извлекаем JSON из ответа (на случай, если LLM обернул его в ```
        json_match = re.search(r'```(?:json)?\s*({.*?})\s*```', raw_response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
        else:
            # Если не найдены маркеры кода, пробуем распарсить весь ответ как JSON
            json_str = raw_response_text.strip()

        # Парсим JSON
        evaluation_result = json.loads(json_str)
        return evaluation_result
    except json.JSONDecodeError as e:
        print(f"❌ Ошибка парсинга JSON из ответа LLM: {e}")
        print(f"Ответ LLM: {raw_response_text}")
        # --- ИСПРАВЛЕНО: возвращаем осмысленную relevance при ошибке ---
        # Попробуем извлечь relevance из raw_response_text, если возможно
        relevance_match = re.search(r'"relevance"\s*:\s*([0-1](?:\.\d+)?)', raw_response_text)
        try:
            relevance_val = float(relevance_match.group(1)) if relevance_match else 0.0
        except (AttributeError, ValueError):
            relevance_val = 0.0

        return {
            "verdict": "ошибка парсинга",
            "feedback": f"Ошибка при анализе ответа: {str(e)}. Не удалось распознать JSON.",
            "relevance": relevance_val # Используем извлечённое значение или 0.0
        }
    except Exception as e:
        print(f"❌ Непредвиденная ошибка при оценке: {e}")
        # Важно: этот блок ловит ВСЕ остальные ошибки, включая NameError изнутри except json.JSONDecodeError
        # Если ошибка произошла в блоке except json.JSONDecodeError, relevance_val может быть не определена
        # Лучше избегать этого, но на всякий случай:
        try:
            relevance_val # Проверим, определена ли она
        except NameError:
            relevance_val = 0.0 # Если нет, используем 0.0
        return {
            "verdict": "ошибка",
            "feedback": f"Непредвиденная ошибка: {str(e)}",
            "relevance": relevance_val,
        }

## API модель

In [ ]:
# --- 6. API-модели данных ---
class QuestionResponse(BaseModel):
    question: str

class EvaluateRequest(BaseModel):
    question: str
    student_answer: str

class EvaluateResponse(BaseModel):
    result: dict

class AnswerResponse(BaseModel): # Оставим старую модель для совместимости, если нужна
    question: str
    answer: str
    context: List[str]

class QuestionRequest(BaseModel): # Оставим старую модель для совместимости, если нужна
    question: str


In [ ]:
# --- 7. Эндпоинты ---
@app.post("/get_question", response_model=QuestionResponse)
async def get_question():
    """Возвращает случайный вопрос из списка."""
    if not questions_list:
        raise HTTPException(status_code=404, detail="Список вопросов пуст.")
    # --- ИСПРАВЛЕНО: выбираем случайный вопрос ---
    question = random.choice(questions_list) # <-- ИЗМЕНЕНО
    # current_question_index += 1 # Больше не нужно
    return QuestionResponse(question=question)

@app.post("/evaluate_answer", response_model=EvaluateResponse)
async def evaluate_answer_endpoint(request: EvaluateRequest):
    """Оценивает ответ студента на заданный вопрос."""
    # Вызываем функцию оценки, передавая retriever
    result = evaluate_answer(request.question, request.student_answer, retriever)
    return EvaluateResponse(result=result)

## Локальный запуск

In [ ]:
# --- 9. Запуск сервера (в Colab) и получение URL ---
# Применяем nest_asyncio для корректной работы в Jupyter/Colab
nest_asyncio.apply()

# def run_fastapi():
#     """Функция для запуска uvicorn в отдельном потоке."""
#     uvicorn.run(app, host="0.0.0.0", port=8001)

def run_fastapi():
    try:
        uvicorn.run(app, host="0.0.0.0", port=8001)
    except OSError as e:
        print(f"[WARN] Сервер уже запущен: {e}")
    except SystemExit:
        print("[WARN] Uvicorn попытался завершить процесс — игнорируем.")

# Запускаем сервер в фоне
import threading
thread = threading.Thread(target=run_fastapi)
thread.start()

public_url_str = "http://0.0.0.0:8001"
print(f"🌍 FastAPI сервер доступен по адресу: {public_url_str}")

🌍 FastAPI сервер доступен по адресу: http://0.0.0.0:8001


## Демонстрация

In [ ]:
# --- 9.1 Демонстрация (опционально) - Интерактивный режим ---
import requests
import json

api_url = public_url_str # Теперь это строка
print(f"DEBUG: Используемый API URL: {api_url}") # Для отладки
print("\n--- Интерактивный режим оценки ---")
print("Введите 'exit' (без кавычек) в качестве ответа, чтобы выйти из режима.")

while True:
    try:
        # 1. Получить вопрос
        q_resp = requests.post(f"{api_url}/get_question", json={})
        if q_resp.status_code != 200:
            print(f"❌ Ошибка при получении вопроса: {q_resp.status_code}, {q_resp.text}")
            break # Выходим при ошибке получения вопроса

        question_data = q_resp.json()
        question = question_data.get('question', 'Вопрос не получен.')
        if not question.strip():
             print("❌ Получен пустой вопрос. Проверьте список вопросов.")
             break

        print(f"\n❓ Вопрос: {question}")

        # 2. Получить ответ от пользователя
        student_answer = input("\nВведите ваш ответ (или 'exit' для выхода): ").strip()

        # 3. Проверить команду выхода
        if student_answer.lower() == 'exit':
            print("👋 Выход из интерактивного режима.")
            break

        # 4. Оценить ответ
        eval_resp = requests.post(f"{api_url}/evaluate_answer", json={
            "question": question,
            "student_answer": student_answer
        })

        if eval_resp.status_code != 200:
            print(f"❌ Ошибка при оценке: {eval_resp.status_code}, {eval_resp.text}")
            continue # Переходим к следующему вопросу, даже если оценка не удалась

        eval_result = eval_resp.json()
        result_data = eval_result.get('result', {})

        print(f"\n--- Результат оценки ---")
        print(f"Вердикт: {result_data.get('verdict', 'Неизвестно')}")
        print(f"Обратная связь: {result_data.get('feedback', 'Отсутствует')}")
        print(f"Релевантность (0-1): {result_data.get('relevance', 0.0)}")

    except KeyboardInterrupt:
        print("\n\n👋 Прервано пользователем. Выход из интерактивного режима.")
        break
    except Exception as e:
        print(f"❌ Непредвиденная ошибка: {e}")
        # В случае ошибки, можно либо выйти, либо продолжить цикл.
        # Лучше продолжить, чтобы пользователь мог попробовать снова.
        continue

print("\n--- Конец интерактивного режима ---")

DEBUG: Используемый API URL: http://0.0.0.0:8001

--- Интерактивный режим оценки ---
Введите 'exit' (без кавычек) в качестве ответа, чтобы выйти из режима.
INFO:     127.0.0.1:47130 - "POST /get_question HTTP/1.1" 200 OK

❓ Вопрос: Как выбрать компромисс между Consistency, Availability и Partition tolerance?

Введите ваш ответ (или 'exit' для выхода): В распределенных системах выбор между согласованностью (C), доступностью (A) и устойчивостью к разделению (P) зависит от бизнес-приоритетов: если точность данных критична (банковские операции), жертвуют доступностью (CP); если система должна быть всегда онлайн (соцсети), допускают временную несогласованность (AP); а в идеальных сетевых условиях можно выбрать CA, но на практике в реальных распределенных системах всегда жертвуют либо C, либо A при возникновении сетевых разделений (P). Ключевой компромисс — определить, что важнее: мгновенная точность данных или непрерывная работа системы.


/tmp/ipython-input-2598013041.py:33: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever_instance.get_relevant_documents(query)


DEBUG: Raw LLM response:
{
  "verdict": "верно",
  "feedback": "Студент правильно понял принцип выбора компромисса между свойствами CAP согласно представленному контексту. Его рассуждения соответствуют изложенным рекомендациям: важность точности данных против доступности системы в зависимости от типа приложения.",
  "relevance": 1.0
}
---
INFO:     127.0.0.1:35754 - "POST /evaluate_answer HTTP/1.1" 200 OK

--- Результат оценки ---
Вердикт: верно
Обратная связь: Студент правильно понял принцип выбора компромисса между свойствами CAP согласно представленному контексту. Его рассуждения соответствуют изложенным рекомендациям: важность точности данных против доступности системы в зависимости от типа приложения.
Релевантность (0-1): 1.0
INFO:     127.0.0.1:35758 - "POST /get_question HTTP/1.1" 200 OK

❓ Вопрос: Назовите основные типы метрик и их назначение.

Введите ваш ответ (или 'exit' для выхода): какие-то метрики
DEBUG: Raw LLM response:
{
  "verdict": "неверно",
  "feedback": "Студент н